### 抓取步道資料並存檔

In [ ]:
# # 抓取步道資料
# import requests
# from bs4 import BeautifulSoup
# import collections
# data = collections.defaultdict(dict)

# i = 1
# while i < 2000:
#     url = f'https://hiking.biji.co/index.php?q=trail&act=detail&id={i}'
#     re = requests.get(url)
#     soup = BeautifulSoup(re.text, "html.parser")
#     title = soup.find('h1', {'class': "text-3xl font-bold"})
#     info = soup.find_all('dd', {'class': "flex-1 p-4"})
#     try:
#         if info[0].text == '香港' or info[0].text == '西班牙':
#             i += 1
#             continue
#     except IndexError:
#         i+=1
#         continue
#     if info[0].text == '香港' or info[0].text == '西班牙':
#         continue
#     try:
#         print(title.text)
#         data[title.text]['所在縣市'] = info[0].text
#         data[title.text]['里程'] = info[1].text
#         data[title.text]['步道類型'] = info[2].text
#         data[title.text]['步道型態'] = info[3].text
#         data[title.text]['海拔高度'] = info[4].text
#         data[title.text]['高度落差'] = info[5].text
#         data[title.text]['路面狀況'] = info[6].text
#         data[title.text]['所需時間'] = info[7].text
#         data[title.text]['難易度'] = info[8].text
#         # print(f'所在縣市:{info[0].text}')
#         # print(f'里程:{info[1].text}')
#         # print(f'步道類型:{info[2].text}')
#         # print(f'步道型態:{info[3].text}')
#         # print(f'海拔高度:{info[4].text}')
#         # print(f'高度落差:{info[5].text}')
#         # print(f'路面狀況:{info[6].text}')
#         # print(f'所需時間:{info[7].text}')
#         # print(f'難易度:{info[8].text}')
#         if info[9].text != '-':
#             data[title.text]['所屬園區'] = info[9].text
#             # print(f'所屬園區:{info[9].text}')
#         if info[10].text != '-':
#             data[title.text]['山系'] = info[10].text
#             # print(f'山系:{info[10].text}')
#         data[title.text]['最適季節'] = info[11].text
#         # print(f'最適季節:{info[11].text}')
#         apply_m = info[12].text.replace(' ', '').replace('\n', '')
#         apply_p = info[13].text.replace(' ', '').replace('\n', '')
#         data[title.text]['申請入山'] = apply_m
#         # print(f'申請入山:{apply_m}')
#         if info[9].text != '-':
#             if apply_p == '否':
#                 data[title.text]['申請入園'] = '否'
#                 # print(f'申請入園:否')
#             else:
#                 data[title.text]['申請入園'] = '是'
#                 # print(f'申請入園:是')
#         print('')
#     except AttributeError:
#         i += 1
#         continue
#     i += 1


In [ ]:
# ## 將抓下來的資料存檔：'mountain_data.csv'
# import pandas as pd
# df = pd.DataFrame.from_dict({(i): dict(data)[i]
#                            for i in dict(data).keys()},
#                        orient='index')
# # df.to_csv('mountain_data.csv')

### 抓取天氣資料

In [ ]:
import requests
import json
from bs4 import BeautifulSoup

# 此函式輸入地名'台'須寫為'臺'
def weather_data(location):

    url = 'https://opendata.cwb.gov.tw/api/v1/rest/datastore/F-C0032-001'
    params = {
        'Authorization': 'CWB-9E6E7429-A9A0-46B1-88AA-CF702E6934B6',
        'locationName': location,
    }

    response = requests.get(url, params=params)
    # print(response.status_code)

    if response.status_code == 200:
        # print(response.text)
        data = json.loads(response.text)

        location = data['records']['location'][0]['locationName']

        weather_elements = data['records']['location'][0]['weatherElement']
        date = weather_elements[0]['time'][0]['startTime'][:10]
        start_time = weather_elements[0]['time'][0]['startTime'][-8:]
        end_time = weather_elements[0]['time'][0]['endTime'][-8:]
        weather_state = weather_elements[0]['time'][0]['parameter']['parameterName']
        rain_prob = weather_elements[1]['time'][0]['parameter']['parameterName']
        min_tem = weather_elements[2]['time'][0]['parameter']['parameterName']
        comfort = weather_elements[3]['time'][0]['parameter']['parameterName']
        max_tem = weather_elements[4]['time'][0]['parameter']['parameterName']

        # print('地點：', location)
        # print('日期:', date)
        # print('時間：',str(start_time) + '-' + str(end_time))
        # print('天氣：', weather_state)
        # print('降雨機率：', rain_prob)
        # print('最高溫度：', min_tem)
        # print('最低溫度：', max_tem)
        # print('舒適度：', comfort)
        return location, date, str(start_time) + '-' + str(end_time), weather_state, rain_prob, min_tem, max_tem, comfort


    else:
        return None

# print(weather_data())

### 載入步道資料

In [ ]:
import pandas as pd
df = pd.read_csv('mountain_data.csv')

FileNotFoundError: ignored

In [ ]:
df

將步道名稱設為index，把 NaN 變成'無'

In [ ]:
df2 = df.set_index('Unnamed: 0')
df2.fillna('無', inplace=True)

將時間轉到對應 '1小時以下','1-2小時','2-4小時','4小時以上' 四個時間區段，並存到'class_list'

In [ ]:
class_list = []
ref_list = df['所需時間'].values.tolist().copy()
for item in ref_list:
  item = item.replace(' ','').replace('(往返)','')
  if '天' in item:
    class_list.append('4小時以上')
  else:
    if '小時' not in item:
      class_list.append('1小時以下')
    else:
      H = int(item.split('小時')[0])
      if H>=1 and H<2:
        class_list.append('1~2小時')
      elif H>=2 and H<4:
        class_list.append('2~4小時')
      elif H>=4:
        class_list.append('4小時以上')

### 測試

In [ ]:
# type_1 = True
# type_c = True
# type_2 = False
# time = '1~2小時'
# application = '否'
# easy ='低'
# place = '台北市'

# if type_1:
#   type1_list = df.iloc[:,0][list(map(lambda x: x=='必須折返',df['步道型態'].values))].tolist()
# else:
#   type1_list = []
# if type_c:
#   typec_list = df.iloc[:,0][list(map(lambda x: x=='環狀',df['步道型態'].values))].tolist()
# else:
#   typec_list = []
# if type_2:
#   type2_list = df.iloc[:,0][list(map(lambda x: x=='雙向進出',df['步道型態'].values))].tolist()
# else:
#   type2_list = []
# type_list =  type2_list + typec_list + type1_list
# # 所需時間
# is_time_list = df.iloc[:,0][list(map(lambda x: x==time,class_list))].tolist()
# # 申請入山
# application_or_not_list = df.iloc[:,0][list(map(lambda x: x==application,df['申請入山'].values))].tolist()
# # 難易度
# easy_hard_list = df.iloc[:,0][list(map(lambda x: x==easy,df['難易度'].values))].tolist()
# # 所在縣市
# place_list = df.iloc[:,0][list(map(lambda x: x[:3]==place,df['所在縣市'].values))].tolist()

# union_list = list(set(place_list) & set(easy_hard_list) & set(type_list) & set(is_time_list) & set(application_or_not_list))



### 連接 Anvil

連接

In [ ]:
!pip install anvil-uplink

In [ ]:
import anvil.server

In [ ]:
anvil.server.connect("JCGNZVCRESJZ55NJUV5LR6PK-XHDA7WOZDBWIYGZF")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


對傳入的值做運算後輸出

In [ ]:
# 推薦步道
@anvil.server.callable
def search(time,type_1,type_2,type_c,place,easy,application):
  # 步道型態
  if type_1:
    type1_list = df.iloc[:,0][list(map(lambda x: x=='必須折返',df['步道型態'].values))].tolist()
  else:
    type1_list = []
  if type_c:
    typec_list = df.iloc[:,0][list(map(lambda x: x=='環狀',df['步道型態'].values))].tolist()
  else:
    typec_list = []
  if type_2:
    type2_list = df.iloc[:,0][list(map(lambda x: x=='雙向進出',df['步道型態'].values))].tolist()
  else:
    type2_list = []
  type_list =  type2_list + typec_list + type1_list
  # 所需時間
  is_time_list = df.iloc[:,0][list(map(lambda x: x==time,class_list))].tolist()
  # 申請入山
  application_or_not_list = df.iloc[:,0][list(map(lambda x: x==application,df['申請入山'].values))].tolist()
  # 難易度
  easy_hard_list = df.iloc[:,0][list(map(lambda x: x==easy,df['難易度'].values))].tolist()
  # 所在縣市
  place_list = df.iloc[:,0][list(map(lambda x: x[:3]==place,df['所在縣市'].values))].tolist()

  union_list = list(set(place_list) & set(easy_hard_list) & set(type_list) & set(is_time_list) & set(application_or_not_list))

  filter_dict = df2.loc[union_list].to_dict('records')
  filter_list = []
  for j in range(len(filter_dict)):
    filter_dict[j]['步道'] = union_list[j]
    filter_list.append(filter_dict[j])
  # if union_list !=[]:
  #   txt = ', '.join(i for i in union_list)
  # else:
  #   txt = '查無資訊！'
  return filter_list

# 搜尋指定縣市天氣
@anvil.server.callable
def search_weather(place):
  if '台' in place:
    place = place.replace('台','臺')
  if weather_data(place) is not None:
    location, date, time, weather_state, rain_prob, min_tem, max_tem, comfort = weather_data(place)
  else:
    weather_value_txt ='查無資訊'
    weather_name_txt =''
  nl = '\n'
  # weather_name_txt = f'地點：{location}{nl}日期：{date}{nl}時間：{time}{nl}天氣：{weather_state}{nl}天氣：{weather_state}{nl}降雨機率：{rain_prob}%{nl}最高溫度：{min_tem}˚C{nl}最低溫度：{max_tem}{nl}˚C舒適度：{comfort}{nl}'
  weather_value_txt = f'{location}{nl}{date}{nl}{time}{nl}{weather_state}{nl}{weather_state}{nl}{rain_prob}%{nl}{min_tem}˚C{nl}{max_tem}˚C{nl}{comfort}'
  weather_name_txt = f'地點：{nl}日期：{nl}時間：{nl}天氣：{nl}天氣：{nl}降雨機率：{nl}最高溫度：{nl}最低溫度：{nl}舒適度：'

  return weather_value_txt,weather_name_txt

讓 anvil 視窗處於持續接收訊息狀態

In [ ]:
anvil.server.wait_forever()